In [44]:
# === Imports ===
import numpy as np
from scipy.optimize import minimize, brentq
from scipy.integrate import quad
from numpy.fft import fft
import pandas as pd
import cmath
from tqdm import tqdm
from scipy.stats import norm
from flask import Flask, request, jsonify

In [45]:
def load_market_data(filepath, S0):
    df = pd.read_csv(filepath)
    market_data = []

    for _, row in df.iterrows():
        K = row["K/S"] * S0
        T = row["T"]
        vol = row["implied_vol"] / 100  # en décimal
        market_data.append((K, T, vol))

    return market_data

In [46]:
# === Paramétrage des modèles ===

def heston_params():
    return {
        "kappa": 1.12,     # speed of mean reversion
        "theta": 0.05,    # variance de long terme
        "sigma": 0.5,     # volatilité de la variance (vol-of-vol)
        "rho": -0.25,      # corrélation entre actif et variance
        "v0": 0.08,        # variance initiale
        "lambda_": 0
    }

def bates_params():
    return {
        "kappa": 1.1,
        "theta": 0.05,
        "sigma": 0.5,
        "rho": -0.25,
        "v0": 0.08,
        "lambda_": 0.3,    # intensité des sauts
        "muJ": 0.02,      # moyenne des sauts log-normaux
        "sigmaJ": 0.2     # écart-type des sauts
    }

def double_heston_params():
    return {
        "kappa1": 1.12,
        "theta1": 0.05,
        "sigma1": 0.5,
        "rho1": -0.25,
        "v01": 0.08,
        "kappa2": 1.25,
        "theta2": 0.06,
        "sigma2": 0.65,
        "rho2": -0.85,
        "v02": 0.04
    }


In [47]:
# Construction des 3 modèles

#Parameters
config = {
    "S0": 100,
    "r": 0.02,
    "q": 0.01
}

# HESTON
def heston_cf(u, T, heston_params, S0, r, q):

    kappa = heston_params["kappa"]
    theta = heston_params["theta"]
    sigma = heston_params["sigma"]
    rho   = heston_params["rho"]
    v0    = heston_params["v0"]
    lambda_ = heston_params["lambda_"]

    # Commun à phi1 et phi2
    x = np.log(S0) #changement de variable pour faciliter l'écriture

    #pour phi1
    u_shift = 0.5
    b1 = kappa + lambda_ - rho*sigma

    d1 = np.sqrt((rho * sigma * 1j * u - b1)**2 - sigma**2 * (2 * u_shift * u - u**2))
    g1 = (b1 - rho*sigma*1j*u + d1) / (b1 - rho*sigma*1j*u - d1)

    C1 = (r-q) * u * 1j * T + (kappa * theta / sigma**2) * ((b1 - rho*sigma*1j*u - d1)*T - 2 * np.log((1 - g1*np.exp(d1*T)) / (1 - g1)))
    D1 = ((b1 - rho * sigma * 1j * u + d1) / sigma**2) * ((1 - np.exp(d1 * T)) / (1 - g1 * np.exp(d1 * T)))

    heston_phi1 = np.exp(C1 + D1 * v0 + 1j * u * (x - q * T))

    #Pour phi2
    u_shift = -0.5
    b2 = kappa + lambda_

    d2 = np.sqrt((rho * sigma * 1j * u - b1)**2 - sigma**2 * (2 * u_shift * u - u**2))
    g2 = (b2 - rho*sigma*1j*u + d2) / (b2 - rho*sigma*1j*u - d2)

    C2 = (r-q) * u * 1j * T + (kappa * theta / sigma**2) * ((b2 - rho*sigma*1j*u - d2)*T - 2 * np.log((1 - g2*np.exp(d2*T)) / (1 - g2)))
    D2 = ((b2 - rho * sigma * 1j * u + d2) / sigma**2) * ((1 - np.exp(d2 * T)) / (1 - g2 * np.exp(d2 * T)))

    heston_phi2 = np.exp(C2 + D2 * v0 + 1j * u * (x - q * T))

    return heston_phi1, heston_phi2


# BATES
def bates_cf(u, T, bates_params, S0=config["S0"], r=config["r"], q=config["q"]):

    heston_phi1, heston_phi2 = heston_cf(u, T, bates_params, S0, r, q)

    # jump parameters
    lambda_ = bates_params["lambda_"]
    muJ = bates_params["muJ"]
    sigmaJ = bates_params["sigmaJ"]

    jump_cf = np.exp(lambda_ * T * (np.exp(1j * u * muJ - 0.5 * sigmaJ**2 * u**2) - 1))

    bates_phi1 = heston_phi1 * jump_cf
    bates_phi2 = heston_phi2 * jump_cf

    return bates_phi1, bates_phi2

# DOUBLE HESTON
def double_heston_cf(u, T, double_heston_params, S0=config["S0"], r=config["r"], q=config["q"]):

    #Extraction des paramètres depuis le dictionnaire
    kappa1 = double_heston_params["kappa1"]
    theta1 = double_heston_params["theta1"]
    sigma1 = double_heston_params["sigma1"]
    rho1   = double_heston_params["rho1"]
    v01    = double_heston_params["v01"]

    kappa2 = double_heston_params["kappa2"]
    theta2 = double_heston_params["theta2"]
    sigma2 = double_heston_params["sigma2"]
    rho2   = double_heston_params["rho2"]
    v02    = double_heston_params["v02"]

    x = np.log(S0)
    u_shift = 1

    a1 = kappa1*theta1
    b1 = kappa1
    a2 = kappa2*theta2
    b2 = kappa2

    #Pour phi1

    d1 = np.sqrt((rho1*sigma1*(u+u_shift)*1j - b1)**2 + sigma1**2 * (1j* (u+u_shift) + (u+u_shift)**2))
    g1 = (b1 - rho1*sigma1*(u+u_shift)*1j + d1)/(b1 - rho1*sigma1*(u+u_shift)*1j - d1)
    
    d2 = np.sqrt((rho2*sigma2*(u+u_shift)*1j - b2)**2 + sigma2**2 * (1j* (u+u_shift) + (u+u_shift)**2))
    g2 = (b2 - rho2*sigma2*(u+u_shift)*1j + d2)/(b2 - rho2*sigma2*(u+u_shift)*1j - d2)

    A = r*(u+u_shift)*1j*T + (a1/(sigma1**2))*((b1 - rho1*sigma1*(u+u_shift)*1j + d1) * T - 2 * np.log((1 - g1 * np.exp(d1*T))/(1 - g1))) + (a2/sigma2**2)*((b2 - rho2*sigma2*(u+u_shift)*1j + d2) * T - 2 * np.log((1 - g2 * np.exp(d2*T))/(1 - g2)))
    B1 = ((b1 - rho1 * sigma1 * (u+u_shift) *1j + d1) / (sigma1**2)) * ((1 - np.exp(d1*T))/(1 - g1*np.exp(d1*T)))
    B2 = ((b2 - rho2 * sigma2 * (u+u_shift) *1j + d2) / (sigma2**2)) * ((1 - np.exp(d2*T))/(1 - g2*np.exp(d2*T)))

    double_heston_phi1 = np.exp(A + B1 * v01 + B2 * v02 + 1j*u*(x - q*T))

    #Pour phi2

    d1 = np.sqrt((rho1*sigma1*u*1j - b1)**2 + sigma1**2 * (1j* u + u**2))
    g1 = (b1 - rho1*sigma1*u*1j + d1)/(b1 - rho1*sigma1*u*1j - d1)
    
    d2 = np.sqrt((rho2*sigma2*u*1j - b2)**2 + sigma2**2 * (1j* u + u**2))
    g2 = (b2 - rho2*sigma2*u*1j + d2)/(b2 - rho2*sigma2*u*1j - d2)

    A = r*u*1j*T + (a1/(sigma1**2))*((b1 - rho1*sigma1*u*1j + d1) * T - 2 * np.log((1 - g1 * np.exp(d1*T))/(1 - g1))) + (a2/sigma2**2)*((b2 - rho2*sigma2*u*1j + d2) * T - 2 * np.log((1 - g2 * np.exp(d2*T))/(1 - g2)))
    B1 = ((b1 - rho1 * sigma1 * u *1j + d1) / (sigma1**2)) * ((1 - np.exp(d1*T))/(1 - g1*np.exp(d1*T)))
    B2 = ((b2 - rho2 * sigma2 * u *1j + d2) / (sigma2**2)) * ((1 - np.exp(d2*T))/(1 - g2*np.exp(d2*T)))

    double_heston_phi2 = np.exp(A + B1 * v01 + B2 * v02 + 1j*u*(x - q*T))

    return double_heston_phi1, double_heston_phi2




In [48]:
u = 5
T = 0.1
S0 = 100
r = 0.02
q = 0.01
params1 = heston_params()
params2 = bates_params()
params3 = double_heston_params()

# Calcul des φ1 et φ2
heston_phi1, heston_phi2 = heston_cf(u, T, params1, S0, r, q)
bates_phi1, bates_phi2 = bates_cf(u, T, params2, S0, r, q)
double_phi1, double_phi2 = double_heston_cf(u, T, params3, S0, r, q)

# Affichage
print("Heston_phi1 =", heston_phi1)
print("Heston_phi2=", heston_phi2)
print("Bates_phi1=",bates_phi1)
print("Bates_phi2=", bates_phi2)
print("Double_phi1=", double_phi1)
print("Double_phi2=", double_phi2)


Heston_phi1 = (-0.43077864214393446-0.7062685086700203j)
Heston_phi2= (-0.4032969309592188-0.6627615750740725j)
Bates_phi1= (-0.4241179075474028-0.6952403523710073j)
Bates_phi2= (-0.3973773125974726-0.653076533634138j)
Double_phi1= (-0.4201033650024569-0.6886515930961374j)
Double_phi2= (-0.45063944991373545-0.7334403803936818j)


In [49]:
def compute_Pj(phi_j, S0, K): #ne marche que pour heston et bates
    
    x = np.log(K)

    def integrand(u):
        numerator = np.exp(-1j * u * x) * phi_j(u)
        denominator = 1j * u
        return np.real(numerator / denominator)

    integral, _ = quad(integrand, 1e-5, 500)
    return 0.5 + (1 / np.pi) * integral


In [50]:
def compute_Pj_double_heston(phi_j, S0, K):
    
    def integrand(u):
        numerator = np.exp(1j * u * np.log(S0/K)) * phi_j(u)
        denominator = 1j * u
        return np.real(numerator / denominator)

    integral, _ = quad(integrand, 1e-5, 500, limit = 200)
    return 0.5 + (1 / np.pi) * integral

In [51]:
S0 = 100
K = 65
T = 0.1
r = 0.02
q = 0.01

params_heston = heston_params()
phi1_h, phi2_h = heston_cf(u, T, params_heston, S0, r, q)

P1_heston = compute_Pj(lambda u: heston_cf(u, T, params_heston, S0, r, q)[0], S0, K)
P2_heston = compute_Pj(lambda u: heston_cf(u, T, params_heston, S0, r, q)[1], S0, K)

params_bates = bates_params()
phi1_b, phi2_b = bates_cf(u, T, params_bates, S0, r, q)

P1_bates = compute_Pj(lambda u: bates_cf(u, T, params_bates, S0, r, q)[0], S0, K)
P2_bates = compute_Pj(lambda u: bates_cf(u, T, params_bates, S0, r, q)[1], S0, K)

params_dh = double_heston_params()
phi1_dh, phi2_dh = double_heston_cf(u, T, params_dh, S0, r, q)

P1_dh = compute_Pj_double_heston(lambda u: double_heston_cf(u, T, params_dh, S0, r, q)[0], S0, K)
P2_dh = compute_Pj_double_heston(lambda u: double_heston_cf(u, T, params_dh, S0, r, q)[1], S0, K)

print(f"Heston : P1 = {P1_heston:.6f}, P2 = {P2_heston:.6f}")
print(f"Bates  : P1 = {P1_bates:.6f}, P2 = {P2_bates:.6f}")
print(f"Double : P1 = {P1_dh:.6f}, P2 = {P2_dh:.6f}")

Heston : P1 = 0.972201, P2 = 0.958540
Bates  : P1 = 0.967103, P2 = 0.953796
Double : P1 = 0.984704, P2 = 0.999984


In [52]:
def pricing_function(S0, r, T, K , P1, P2):
    C = max(S0 * P1 - K*np.exp(-r*T)*P2, 0)
    return C


In [53]:
S0 = 100
K = 65
T = 0.1
r = 0.02

print("Pricing Heston :", pricing_function(S0, r , T , K , P1_heston, P2_heston))
print("Pricing Bates :", pricing_function(S0, r , T , K , P1_bates, P2_bates))
print("Princing Double Heston :", pricing_function(S0, r , T , K , P1_dh, P2_dh))
print("Bid/Ask Spread =", max(pricing_function(S0, r , T , K , P1_heston, P2_heston),pricing_function(S0, r , T , K , P1_bates, P2_bates),pricing_function(S0, r , T , K , P1_dh, P2_dh))-min(pricing_function(S0, r , T , K , P1_heston, P2_heston),pricing_function(S0, r , T , K , P1_bates, P2_bates),pricing_function(S0, r , T , K , P1_dh, P2_dh)))


Pricing Heston : 35.03947500429057
Pricing Bates : 34.83746531068272
Princing Double Heston : 33.60135005778376
Bid/Ask Spread = 1.4381249465068109


In [54]:
# ============= CALIBRAGE ==============

# === Paramètres de marché ===
S0 = 100
r = 0.02
q = 0.01

# === Chargement complet du fichier Excel ===
df_market = pd.read_excel("NDX-2014_clean-3.xlsx", header=1)

market_data_sample = []
for _, row in df_market.iterrows():
    K = row["K/S"] * S0
    T = row["T"]
    vol = row["implied_vol"] / 100  # en décimal
    market_data_sample.append((K, T, vol))

market_data_sample = market_data_sample[:20] #pour limiter l'univers de calcul

# === Black-Scholes Call avec dividende ===
def black_scholes_call(S0, K, T, r, q, sigma):
    d0 = 1 / (sigma * np.sqrt(T)) * np.log(S0 / (K * np.exp(-(r - q) * T))) - 0.5 * sigma * np.sqrt(T)
    d1 = d0 + sigma * np.sqrt(T)
    return S0 * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d0)

# === Inversion de la vol implicite
def implied_vol(target_price, S0, K, T, r, q):
    try:
        return brentq(lambda sigma: black_scholes_call(S0, K, T, r, q, sigma) - target_price, 1e-6, 5.0, maxiter=500)
    except:
        return np.nan


# === Fonction de coût pour la calibration Heston
def calibrate_error_heston(params_vec, market_data, S0, r, q):
    kappa, log_theta, sigma, rho, v0 = params_vec
    theta = np.exp(log_theta)
    model_params = {
        "kappa": kappa, "theta": theta, "sigma": sigma,
        "rho": rho, "v0": v0, "lambda_": 0
    }

    errors = []
    for K, T, market_vol in market_data:
        try:
            phi1 = lambda u: heston_cf(u, T, model_params, S0, r, q)[0]
            phi2 = lambda u: heston_cf(u, T, model_params, S0, r, q)[1]
            P1 = compute_Pj(phi1, S0, K)
            P2 = compute_Pj(phi2, S0, K)
            model_price = pricing_function(S0, r, T, K, P1, P2)
            implied = implied_vol(model_price, S0, K, T, r, q)
            if not np.isnan(implied):
                errors.append((implied - market_vol) ** 2)
        except:
            continue

    return np.mean(errors)

# === Paramètres initiaux et bornes
x0 = [1.0, np.log(0.04), 0.3, -0.5, 0.04]
bounds = [
    (0.01, 5),                     # kappa
    (np.log(0.01), np.log(0.5)),   # log_theta
    (0.01, 1),                     # sigma
    (-0.99, 0.99),                 # rho
    (0.01, 1)                      # v0
]

# === Optimisation
result = minimize(
    fun=lambda p: calibrate_error_heston(p, market_data_sample, S0, r, q),
    x0=x0,
    bounds=bounds,
    method='L-BFGS-B',
    options={
        'disp': False,
        'maxiter': 400,
        'ftol': 1e-6
    }
)
# création des valeurs pour le pricing
kappa_heston_calibrated = result.x[0]
theta_heston_calibrated = np.exp(result.x[1])
sigma_heston_calibrated = result.x[2]
rho_heston_calibrated = result.x[3]
v0_heston_calibrated = result.x[4]


# === Résultats
print("\n✅ Résultat de la calibration HESTON :")
print("kappa  =", result.x[0])
print("theta  =", np.exp(result.x[1]))
print("sigma  =", result.x[2])
print("rho    =", result.x[3])
print("v0     =", result.x[4])
print("Erreur finale :", result.fun)


✅ Résultat de la calibration HESTON :
kappa  = 1.0009581482830343
theta  = 0.04002234771842213
sigma  = 0.29682737883159127
rho    = -0.5000682513365684
v0     = 0.03882877494066037
Erreur finale : 0.01161404962354832


In [55]:
# CALIBRAGE BATES
def calibrate_error_bates(params_vec, market_data, S0, r, q):
    kappa, log_theta, sigma, rho, v0, lambda_, muJ, sigmaJ = params_vec
    theta = np.exp(log_theta)
    b_params = {
        "kappa": kappa, "theta": theta, "sigma": sigma,
        "rho": rho, "v0": v0, "lambda_": lambda_,
        "muJ": muJ, "sigmaJ": sigmaJ
    }
    errors = []
    for K, T, mkt_vol in market_data:
        phi1 = lambda u: bates_cf(u, T, b_params, S0, r, q)[0]
        phi2 = lambda u: bates_cf(u, T, b_params, S0, r, q)[1]
        P1 = compute_Pj(phi1, S0, K)
        P2 = compute_Pj(phi2, S0, K)
        price = max(S0 * P1 - K * np.exp(-r*T) * P2, 0)
        imp = implied_vol(price, S0, K, T, r, q)
        if not np.isnan(imp):
            errors.append((imp - mkt_vol)**2)
    return np.mean(errors)

# Initial guess et bornes pour Bates
x0_bates = [1.1, np.log(0.05), 0.5, -0.25, 0.08, 0.3, 0.02, 0.2]
bounds_bates = [
    (0.01, 5), (np.log(0.01), np.log(0.5)), (0.01, 1), (-0.99, 0.99), (0.001, 1),
    (0.001, 5), (-1, 1), (0.01, 1)
]

# Lancement de la calibration Bates
res_bates = minimize(
    lambda p: calibrate_error_bates(p, market_data_sample, S0, r, q),
    x0=x0_bates,
    bounds=bounds_bates,
    method='L-BFGS-B',
    options={'disp': False, 'maxiter': 400, 'ftol': 1e-6}
)

# création des valeurs pour le pricing
kappa_bates_calibrated   = res_bates.x[0]
theta_bates_calibrated   = np.exp(res_bates.x[1])
sigma_bates_calibrated   = res_bates.x[2]
rho_bates_calibrated     = res_bates.x[3]
v0_bates_calibrated      = res_bates.x[4]
lambda_bates_calibrated  = res_bates.x[5]
muJ_bates_calibrated     = res_bates.x[6]
sigmaJ_bates_calibrated  = res_bates.x[7]


print("\n✅ Calibration Bates terminée :")
print(f"kappa  = {kappa_bates_calibrated:.4f}")
print(f"theta  = {theta_bates_calibrated:.4f}")
print(f"sigma  = {sigma_bates_calibrated:.4f}")
print(f"rho    = {rho_bates_calibrated:.4f}")
print(f"v0     = {v0_bates_calibrated:.4f}")
print(f"lambda = {lambda_bates_calibrated:.4f}")
print(f"muJ    = {muJ_bates_calibrated:.4f}")
print(f"sigmaJ = {sigmaJ_bates_calibrated:.4f}")
print(f"Erreur = {res_bates.fun:.6e}")



✅ Calibration Bates terminée :
kappa  = 1.1192
theta  = 0.0507
sigma  = 0.4581
rho    = -0.2467
v0     = 0.0010
lambda = 0.3121
muJ    = 0.0073
sigmaJ = 0.2130
Erreur = 1.414607e-02


In [ ]:
def calibrate_error_double_heston(params_vec, market_data, S0, r, q):
    kappa1, log_theta1, sigma1, rho1, v01, \
    kappa2, log_theta2, sigma2, rho2, v02 = params_vec

    theta1 = np.exp(log_theta1)
    theta2 = np.exp(log_theta2)
    dh_params = {
        "kappa1": kappa1,   "theta1": theta1,   "sigma1": sigma1,
        "rho1": rho1, "v01": v01,
        "kappa2": kappa2, "theta2": theta2, "sigma2": sigma2,
        "rho2": rho2,"v02": v02
    }

    errors = []
    for K, T, mkt_vol in market_data:
        try:
            phi1 = lambda u: double_heston_cf(u, T, dh_params, S0, r, q)[0]
            phi2 = lambda u: double_heston_cf(u, T, dh_params, S0, r, q)[1]
            P1 = compute_Pj(phi1, S0, K)
            P2 = compute_Pj(phi2, S0, K)
            price = max(S0 * np.exp(-q * T) * P1 - K * np.exp(-r * T) * P2, 0)
            imp = implied_vol(price, S0, K, T, r, q)
            if not np.isnan(imp):
                errors.append((imp - mkt_vol) ** 2)
        except:
            continue

    return np.mean(errors)

# === Paramètres initiaux et bornes ===
x0_double_heston = [1.0, np.log(0.04), 0.3, -0.5, 0.04, 1.5, np.log(0.02), 0.4, -0.3, 0.02]

bounds_double_heston = [
    (0.01, 5),                     # kappa1
    (np.log(0.01), np.log(0.5)),   # log_theta1
    (0.01, 1),                     # sigma1
    (-0.99, 0.99),                 # rho1
    (0.01, 1),                     # v01
    (0.01, 5),                     # kappa2
    (np.log(0.01), np.log(0.5)),   # log_theta2
    (0.01, 1),                     # sigma2
    (-0.99, 0.99),                 # rho2
    (0.01, 1)                      # v02
]

result_dh = minimize(
    lambda p: calibrate_error_double_heston(p, market_data_sample, S0, r, q),
    x0 = x0_double_heston,
    bounds = bounds_double_heston,
    method  = 'L-BFGS-B',
    options = {
        'disp':   False,
        'maxiter': 400,
        'ftol':   1e-6
    }
)

kappa1_double_heston_calibrated = result_dh.x[0]
theta1_double_heston_calibrated = np.exp(result_dh.x[1])
sigma1_double_heston_calibrated = result_dh.x[2]
rho1_double_heston_calibrated   = result_dh.x[3]
v01_double_heston_calibrated    = result_dh.x[4]

kappa2_double_heston_calibrated = result_dh.x[5]
theta2_double_heston_calibrated = np.exp(result_dh.x[6])
sigma2_double_heston_calibrated = result_dh.x[7]
rho2_double_heston_calibrated   = result_dh.x[8]
v02_double_heston_calibrated    = result_dh.x[9]

print("\n✅ Variables calibrées Double Heston :")
print(f"kappa1 = {kappa1_double_heston_calibrated:.4f}")
print(f"theta1 = {theta1_double_heston_calibrated:.4f}")
print(f"sigma1 = {sigma1_double_heston_calibrated:.4f}")
print(f"rho1   = {rho1_double_heston_calibrated:.4f}")
print(f"v01    = {v01_double_heston_calibrated:.4f}")
print(f"kappa2 = {kappa2_double_heston_calibrated:.4f}")
print(f"theta2 = {theta2_double_heston_calibrated:.4f}")
print(f"sigma2 = {sigma2_double_heston_calibrated:.4f}")
print(f"rho2   = {rho2_double_heston_calibrated:.4f}")
print(f"v02    = {v02_double_heston_calibrated:.4f}")
print(f"Erreur finale = {result_dh.fun:.6e}")




✅ Variables calibrées Double Heston :
kappa1 = 1.0001
theta1 = 0.0400
sigma1 = 0.2952
rho1   = -0.4949
v01    = 0.3308
kappa2 = 1.5000
theta2 = 0.0200
sigma2 = 0.3996
rho2   = -0.2965
v02    = 0.3233
Erreur finale = 1.414607e-02


In [57]:
# ======== PRICING HESTON =======

S0 = 100
K = 65
T = 0.1
r = 0.02
q = 0.01

# PARAMS PRICING HESTON
def get_heston_params_calibrated():
    return {
        "kappa": kappa_heston_calibrated,
        "theta": theta_heston_calibrated,
        "sigma": sigma_heston_calibrated,
        "rho":   rho_heston_calibrated,
        "v0":    v0_heston_calibrated,
        "lambda_": 0.0
    }

# Pricing Vanille
def price_heston(S0, K, T, r, q, params):
    phi1 = lambda u: heston_cf(u, T, params, S0, r, q)[0]
    phi2 = lambda u: heston_cf(u, T, params, S0, r, q)[1]
    P1 = compute_Pj(phi1, S0, K)
    P2 = compute_Pj(phi2, S0, K)

    call_price_raw = S0 * np.exp(-q * T) * P1 - K*np.exp(-r * T) * P2
    put_price_raw = K*np.exp(-r * T) * (1-P2) - S0 * np.exp(-q * T) * (1-P1)

    call_price = np.maximum(call_price_raw, 0.0)
    put_price  = np.maximum(put_price_raw,  0.0)

    return call_price, put_price


params = get_heston_params_calibrated()
call_price, put_price = price_heston(S0, K, T, r, q, params)

print(f"Call Heston : {call_price:.4f}")
print(f"Put  Heston : {put_price:.4f}")


Call Heston : 33.9266
Put  Heston : 0.0000


In [58]:
# ==== PRICING BATES ====

S0 = 100
K = 65
T = 0.1
r = 0.02
q = 0.01

# PARAMS PRICING BATES
def get_bates_params_calibrated():
    return {
        "kappa": kappa_bates_calibrated,
        "theta": theta_bates_calibrated,
        "sigma": sigma_bates_calibrated,
        "rho":   rho_bates_calibrated,
        "v0":    v0_bates_calibrated,
        "lambda_": lambda_bates_calibrated,
        "muJ": muJ_bates_calibrated,
        "sigmaJ" : sigmaJ_bates_calibrated
    }

def price_bates(S0, K, T, r, q, params):
    phi1 = lambda u: bates_cf(u, T, params, S0, r, q)[0]
    phi2 = lambda u: bates_cf(u, T, params, S0, r, q)[1]
    P1 = compute_Pj(phi1, S0, K)
    P2 = compute_Pj(phi2, S0, K)

    call_price_raw = S0 * np.exp(-q * T) * P1 - K*np.exp(-r * T) * P2
    put_price_raw = K*np.exp(-r * T) * (1-P2) - S0 * np.exp(-q * T) * (1-P1)

    call_price = np.maximum(call_price_raw, 0.0)
    put_price  = np.maximum(put_price_raw,  0.0)

    return call_price, put_price

params = get_bates_params_calibrated()
call_price, put_price = price_bates(S0, K, T, r, q, params)

print(f"Call Bates : {call_price:.4f}")
print(f"Put  Bates : {put_price:.4f}")



Call Bates : 33.8879
Put  Bates : 0.0000


In [59]:
# ==== PRICING DOUBLE HESTON ====

S0 = 100
K = 65
T = 0.1
r = 0.02
q = 0.01

# PARAMS PRICING DOUBLE HESTON
def get_double_heston_params_calibrated():
    return {
        "kappa1": kappa1_double_heston_calibrated,
        "theta1": theta1_double_heston_calibrated,
        "sigma1": sigma1_double_heston_calibrated,
        "rho1":   rho1_double_heston_calibrated,
        "v01":    v01_double_heston_calibrated,
        "kappa2": kappa2_double_heston_calibrated,
        "theta2": theta2_double_heston_calibrated,
        "sigma2": sigma2_double_heston_calibrated,
        "rho2":   rho2_double_heston_calibrated,
        "v02":    v02_double_heston_calibrated
    }

def price_double_heston(S0, K, T, r, q, params):
    P1 = compute_Pj_double_heston(lambda u: double_heston_cf(u, T, params_dh, S0, r, q)[0], S0, K)
    P2 = compute_Pj_double_heston(lambda u: double_heston_cf(u, T, params_dh, S0, r, q)[1], S0, K)

    print(P1)
    print(P2)

    call_price_raw = S0 * np.exp(-q * T) * P1 - K*np.exp(-r * T) * P2
    put_price_raw = K*np.exp(-r * T) * (1-P2) - S0 * np.exp(-q * T) * (1-P1)

    call_price = np.maximum(call_price_raw, 0.0)
    put_price  = np.maximum(put_price_raw,  0.0)

    return call_price, put_price

params = get_double_heston_params_calibrated()
call_price, put_price = price_double_heston(S0, K, T, r, q, params)

print(f"Call Double Heston : {call_price:.4f}")
print(f"Put  Double Heston : {put_price:.4f}")

0.9847044113817958
0.999983985958064
Call Double Heston : 33.5029
Put  Double Heston : 0.0000


In [70]:
def variance_swap_fair_strike(model_name, T):
    """Retourne le strike du variance swap pour un modèle donné."""

    if model_name == "heston":
        params = get_heston_params_calibrated()
        kappa = params["kappa"]
        theta = params["theta"]
        v0 = params["v0"]
        return theta*T + ((v0-theta)/kappa) * (1 - np.exp(-kappa*T))

    elif model_name == "bates":         # même expression que Heston car les sauts n'affectent pas la variance instantanée
        params = get_bates_params_calibrated()
        kappa = params["kappa"]
        theta = params["theta"]
        v0    = params["v0"]
        lamb = params["lambda_"]
        muJ = params["muJ"]
        sigmaJ = params["sigmaJ"]
        return theta*T + ((v0-theta)/kappa) * (1 - np.exp(-kappa*T)) + lamb*(muJ**2 + sigmaJ**2)

    elif model_name == "double_heston":
        # Somme des deux moteurs de variance
        params = get_double_heston_params_calibrated()

        kappa1 = params["kappa1"]
        theta1 = params["theta1"]
        v01    = params["v01"]

        kappa2 = params["kappa2"]
        theta2 = params["theta2"]
        v02    = params["v02"]

        var1 = theta1*T + ((v01-theta1)/kappa1) * (1 - np.exp(-kappa1*T))
        var2 = theta2*T + ((v02-theta2)/kappa2) * (1 - np.exp(-kappa2*T))
        return var1 + var2

    else:
        raise ValueError(f"Modèle non reconnu : {model_name}")
    

def price_variance_swap(model_name, T, notional, strike_market):
    """
    Retourne la valeur (mark-to-model) du variance swap à l'instant t=0.
    """
    fair_strike = variance_swap_fair_strike(model_name, T)
    value = notional * (fair_strike - strike_market)

    return fair_strike, value


In [71]:
# === Paramètres du swap ===
T = 1.0  # maturité 1 an
notional = 1000000  # en $
strike_market = 0.035  # variance de marché (annualisée, ex. 35%)

# === Test pour chaque modèle ===
for model in ["heston", "bates", "double_heston"]:
    fair_strike, value = price_variance_swap(model, T, notional, strike_market)
    print(f"\n{model.upper()} :")
    print(f"  ↳ Fair strike     : {fair_strike:.6f}")
    print(f"  ↳ Market strike   : {strike_market:.6f}")
    print(f"  ↳ Swap Value      : {value:,.2f} $")


HESTON :
  ↳ Fair strike     : 0.039268
  ↳ Market strike   : 0.035000
  ↳ Swap Value      : 4,268.17 $

BATES :
  ↳ Fair strike     : 0.034976
  ↳ Market strike   : 0.035000
  ↳ Swap Value      : -24.45 $

DOUBLE_HESTON :
  ↳ Fair strike     : 0.400961
  ↳ Market strike   : 0.035000
  ↳ Swap Value      : 365,960.91 $
